In [1]:
import os
import pandas as pd

BASE_DIR = "../../"
RESULTS_DIR = os.path.join(BASE_DIR, "results")

In [2]:
# choose a dataset to analyze
DATASET = 'adult'

In [3]:
df_results = pd.read_csv( os.path.join(RESULTS_DIR, f"results_{DATASET}.csv") )

In [4]:
df_results.head()

,Dataset,Seed,Model_Type,Metric,Val,Hyper_Val,Test,Extra
0,adult,15,fweg,G-mean,0.658075,0.658075,0.687614,"{""groups"": [], ""groups_descr"": ""single_group"",..."
1,adult,15,fweg,G-mean,0.499975,0.499975,0.499976,"{""groups"": [], ""groups_descr"": ""single_group"",..."
2,adult,15,fweg,G-mean,0.658013,0.658013,0.687638,"{""groups"": [], ""groups_descr"": ""single_group"",..."
3,adult,15,fweg,G-mean,0.499975,0.499975,0.499976,"{""groups"": [], ""groups_descr"": ""single_group"",..."
4,adult,15,fweg,G-mean,0.653142,0.653142,0.686309,"{""groups"": [], ""groups_descr"": ""single_group"",..."


In [5]:
df_results = df_results[ df_results['Dataset'] == DATASET ]
df_baselines = df_results[ df_results['Model_Type'] != 'fweg' ]
df_fweg = df_results[ df_results['Model_Type'] == 'fweg' ]

In [6]:
def check_duplicates(df):
    # sometimes you can have multiple results for the same (seed, model_type, extra)
    # combo because you didn't delete old results
    # this identifies that
    t = df[['Seed', 'Model_Type', 'Extra']]
    d = t.drop_duplicates()
    assert t.shape == d.shape
    
check_duplicates(df_fweg)

In [7]:
df_fweg = df_fweg[~df_fweg['Extra'].apply(lambda x: '"groups": 5' in x)]
# separate out FWEG that ran knowing the true form the metric and FWEG that
# ran against a black-box evaluation of the metric
known_metric = df_fweg['Extra'].apply(lambda x: '"FW_val_flag": true' in x)
df_fweg['Model_Type'].loc[known_metric] = 'fweg_known_metric'
df_fweg['Model_Type'].loc[~known_metric] = 'fweg_unknown_metric'

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [8]:
# baselines we can group on all columns but the seed, then perform aggregation
groups_baselines = df_baselines.groupby(['Dataset', 'Model_Type', 'Metric'])
# fweg we need to group by seed first to extract the hyperparameters that performed best
groups_fweg = df_fweg.groupby(['Model_Type', 'Dataset', 'Metric', 'Seed'])

In [9]:
def get_best_fweg(df):
    best = df['Hyper_Val'].idxmax()
    return df.loc[best]

In [10]:
df_best_fweg = groups_fweg.apply(get_best_fweg)
df_best_fweg.reset_index(drop=True, inplace=True)
df_best_fweg.sort_values(['Model_Type', 'Seed', 'Metric'])

In [11]:
df_best_fweg.groupby(['Dataset', 'Model_Type', 'Metric']).mean()

In [12]:
groups_baselines.mean()